In [1]:
import pandas as pd
import numpy as np

from pyiron import Project, ase_to_pyiron
from pyiron_contrib.atomistic.atomicrex.atomicrex_job import Atomicrex

In [2]:
data_pr = Project("../../datasets")
if len(data_pr.job_table()) == 0:
    data_pr.unpack("Cu_training_archive")
data_pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,619,finished,None,df1_A1_A2_A3_EV_elast_phon,/df1_A1_A2_A3_EV_elast_phon,/home/niklas/pyiron/projects/,import_database/Cu_database/,2021-02-08 10:33:52.341472,None,None,zora@cmti001#1,GenericJob,0.4,None,None
1,620,finished,None,df3_10k,/df3_10k,/home/niklas/pyiron/projects/,import_database/Cu_database/,2021-02-08 10:33:53.993230,None,None,zora@cmti001#1,GenericJob,0.4,None,None
2,621,finished,None,df2_1k,/df2_1k,/home/niklas/pyiron/projects/,import_database/Cu_database/,2021-02-08 10:33:54.435308,None,None,zora@cmti001#1,GenericJob,0.4,None,None


In [3]:
data_job = data_pr.load("df1_A1_A2_A3_EV_elast_phon")
df = data_job.to_pandas()

In [4]:
pr = Project("WorkshopPotential")

Are you sure you want to delete all jobs from 'WorkshopPotential'? y/(n) y


In [5]:
job = pr.create_job(Atomicrex, "PotentialDF1")

### Add the structures that should be fitted.
It is possible to assign different weights to certain structures or properties, depending on what should be investigated using the potential. Here every structure has the same weight, but the force vector with N*3 values is normalized to have the same total weight as the single value energy. Therefore it is divided by the number of atoms. 

In [6]:
for id, row in df.iterrows():
    struct = ase_to_pyiron(row.atoms)
    s = job.structures.add_structure(struct, f"id{id}", relative_weight=1)
    s.fit_properties.add_FitProperty("atomic-energy", target_value=row.energy/row.number_of_atoms, relative_weight=1)
    s.fit_properties.add_FitProperty("atomic-forces", target_value=row.forces, relative_weight=1/row.number_of_atoms)

### Define the type of potential and necessary functions.
In this case an eam potential is fitted.

In [7]:
job.potential = job.factories.potentials.eam_potential()

It is necessary to define a pair potential, an electronic density function and an embedding function.
For all of those it is possible to choose between different functional forms.
Classic pair potentials are physically motivated and  have a very limited number of paramaters that are derived from a experimentally measured quantity.
Splines or polynomials offer more flexibility, but are not directly physically motivated and can lead to unphysical oscillations or overfitting.

In this case a generalized morse function is used for the pair interaction, while the electronic density and embedding function will be splines. Depending on the properties that are calculated other functional forms could give better results.

The parameters in D0=3.5 and r0=1.8 are the approximate cohesive energy and the equilibrium lattice constant. Beta and S can also be derived from physical quantities but are chosen randomly in a typical range in this case. Delta is a parameter that shifts the whole function up or down. The initial parameter choices should not matter too much as long as they are somewhat reasonable since they will be optimized in the fitting process anyway.

In [8]:
V = job.factories.functions.morse_B(identifier="V_CuCu", D0=3.5, r0=1.8, beta=2, S=2, delta=0)

In [9]:
V.parameters.D0.min_val = 0
V.parameters.D0.max_val = 5
V.parameters.r0.min_val = 1
V.parameters.r0.max_val = 2.5
V.parameters.delta.min_val = -1
V.parameters.delta.max_val = 1
V.parameters.beta.min_val = 0.1
V.parameters.beta.max_val = 10

Additionally a screening function needs to be defined for the morse potential

In [10]:
V.screening  = job.factories.functions.exp_A_screening(identifier="V_cutoff", cutoff=7)

The electron density is chosen to be a spline function. The cutoff has to be defined. Derivatives left and right are optional, they default to 0. For the right cutoff this is fine, since the forces should smoothly go to 0. For the left this is not necessarily the best choice, since the function value should increase at very close distances.

In [11]:
rho = job.factories.functions.spline(identifier="rho_CuCu", cutoff=7, derivative_left=-1)

For a spline function it is necessary to define node points. They can be equally spaced or sampled with higher density around turning points, f.e. the first neighbor distance.
Too few nodepoints mean low flexibilty, too many lead to overfitting. This requires some testing to find an optimal choice.

In [12]:
rho_nodes = np.linspace(0.5, 7.0, 7).round(2)
rho_nodes

array([0.5 , 1.58, 2.67, 3.75, 4.83, 5.92, 7.  ])

The nodes need initial values. The electron density should be proportional to $e^{-r}$, so this function is chosen to calculate them.

In [13]:
decaying_exp = lambda r: np.exp(-r)
rho_initial = decaying_exp(rho_nodes)

Additionally it is a good idea to define limits for the node points. This is optional for local minimizers, but the fit can quickly run away without limits. Global optimizers typically require them to constrain the sampled space.

A density can't be negative so the lower limit is set to 0. The upper limit is chosen to be 3 times the initial values. These choices aswell as the choice for $e^{-r}$ as initial values are somewhat arbitrary, but don't matter much. The electron density from single atoms does not directly influence the calculated energies and forces, instead the summed up density at some place is used in the embedding function, so the final numerical values are an interplay between electron density and embedding function. Since the latter will also be a spline function it can only be defined for a certain range of rho values as node points. Therefore it is better to limit the range of electron density values and define larger limits for the embedding function instead. 

In [14]:
rho_mins = np.zeros((len(rho_nodes)))
rho_maxs = 3*rho_initial.round(6)
rho.parameters.create_from_arrays(rho_nodes, rho_initial, min_vals=rho_mins, max_vals=rho_maxs)

In [15]:
rho.parameters["node_7.0"].start_val = 0
rho.parameters["node_7.0"].enabled = False

$-\sqrt(\rho)$ can be used as initial guess for the embedding energy, which is taken from second moment approximation tight binding. 
The node points have to be chosen in a range compatible to the electron density. This can be estimated by calculating it for a densely packed structure.
Alternatively atomicrex writes the maximum electron density of all structures to the output. This can be used as a hint for the node points for consequent fits.
Everything else is similar to the electron density.

In [16]:
F = job.factories.functions.spline(identifier="F_CuCu", cutoff=5)
F_nodes = np.linspace(0.0, 5.0, 7).round(2) #9 is worse 11 is worse 7 is best
F_init = -np.sqrt(F_nodes)
F_maxs = np.zeros(len(F_nodes))
F_mins = -np.ones(len(F_nodes))*5
F.parameters.create_from_arrays(F_nodes, F_init, F_mins, F_maxs)
F.parameters["node_0.0"].enabled=False
F.parameters["node_0.0"].start_val = 0
F.parameters["node_5.0"].enabled=False
F.parameters["node_5.0"].start_val = 0

The functions have to be assigned to the potential

In [17]:
job.potential.pair_interactions[V.identifier] = V
job.potential.electron_densities[rho.identifier] = rho
job.potential.embedding_energies[F.identifier] = F

### Define fitting procedure
Finally a few parameters need to be set that influence the fitting process.
The minimization can be done with different algorithms. Atomicrex itself implements the BFGS algorithm. Additionally the algorithms from the nlopt library can be used.

In [18]:
## Define the atom types of the potential
job.input.atom_types.Cu = None
##
job.input.fit_algorithm = job.factories.algorithms.ar_lbfgs(max_iter=500)
job.run()
job.output

The job PotentialDF1 was saved and received the ID: 622


Output({'error': None, 'residual': 0.162222, 'iterations': 19588})

### Same for the 1000 structures dataset
The final parameters of the 100 structure fit can be used for the 1000 Structure fit. This speeds up the fitting process and often leads to better results.

In general it is a good idea to start with few structures and try around with different functions and initial parameters. This is much faster than using all structures from the beginning and gives good guesses for the initial values of the parameters. It also allows to use global optimization with millions of steps in short time spans.

### This can take long and writes 1000 seperate POSCAR files, TAKE CARE

In [19]:
j = pr.create_job(Atomicrex, "PotentialDF2")
j.potential = job.potential.copy()
## Use the final parameters as starting values for the new fit
j.potential.copy_final_to_initial_params()
j.input = job.input.copy()

In [20]:
df = data_pr.load("df2_1k").to_pandas()
for id, row in df.iterrows():
    struct = ase_to_pyiron(row.atoms)
    s = j.structures.add_structure(struct, f"id{id}", relative_weight=1)
    s.fit_properties.add_FitProperty("atomic-energy", target_value=row.energy/row.number_of_atoms, relative_weight=1)
    s.fit_properties.add_FitProperty("atomic-forces", target_value=row.forces, relative_weight=1/row.number_of_atoms)

In [21]:
import time

In [22]:
t1 = time.time()
j.run()
t2 = time.time()

The job PotentialDF2 was saved and received the ID: 623


In [23]:
t2-t1

538.0122790336609

In [24]:
j.output

Output({'error': None, 'residual': 117.878, 'iterations': 3619})

This is the result if the initilly guessed values are taken instead of the fitted ones.

In [25]:
j2 = pr.create_job(Atomicrex, "PotentialDF2_BadStartParams", delete_existing_job=True)
j2.potential = job.potential.copy()
j2.input = j.input.copy()
j2.structures = j.structures
j2.run()

The job PotentialDF2_BadStartParams was saved and received the ID: 624


In [26]:
j2.output

Output({'error': None, 'residual': 8269.14, 'iterations': 607})